## 计算两维的xy的平方根得到一维
*  `feature1 =  torch.sqrt( c[:, 0]**2 +  c[:,1]**2) # 两维的平方根`

In [10]:
import torch

c = torch.tensor([[3,4,5],[6,8,4]], dtype=torch.float)
print(c)

tensor([[3., 4., 5.],
        [6., 8., 4.]])


In [11]:
feature1 =  torch.sqrt( c[:, 0]**2 +  c[:,1]**2) # 两维的平方根
# feature1 =  c[:, 0] + c[:,1]
feature1 = feature1.type_as(c).view(-1, 1) # 维度要一致 (3, 1)
print(feature1)
print()

tensor([[ 5.],
        [10.]])



In [12]:
# c[:, 1] = feature1.squeeze()
c[:, 1] = torch.sqrt( c[:, 0]**2 +  c[:,1]**2) # 两维的平方根
print(c[:, 1:])

tensor([[ 5.,  5.],
        [10.,  4.]])


In [13]:
feature = [c]
feature.append(feature1)
print(feature) # 列表
feature = torch.cat(feature, dim=1) #  最后一维合并Combine together feature decorations
print(feature)

[tensor([[ 3.,  5.,  5.],
        [ 6., 10.,  4.]]), tensor([[ 5.],
        [10.]])]
tensor([[ 3.,  5.,  5.,  5.],
        [ 6., 10.,  4., 10.]])


## torch.randint_like(sample_, low=0, high=num_sample)
https://pytorch.org/docs/stable/generated/torch.randint_like.html


In [14]:
# lmv/core/modules/ops/samples.py
import torch
xyz = torch.rand(12,4)
# print(xyz)
num_sample = 6
valid_sample_ids = torch.arange(12)
print(valid_sample_ids)
# 只要6个
sample_ = xyz.new_zeros(num_sample).long() # 方便的复制原来tensor的所有类型，比如数据类型和数据所在设备等等
print("sample_: ", sample_)
extra_idx = torch.randint_like(sample_, low=0, high=num_sample) # input填充了在low（包括）和 high（不包括）之间均匀生成的随机整数。====================
print("extra_idx: ", extra_idx)
sample_ids = valid_sample_ids[extra_idx]
print(sample_ids)
print(xyz[sample_ids,:])

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
sample_:  tensor([0, 0, 0, 0, 0, 0])
extra_idx:  tensor([1, 0, 4, 2, 4, 0])
tensor([1, 0, 4, 2, 4, 0])
tensor([[0.7889, 0.4091, 0.5140, 0.4285],
        [0.3778, 0.9903, 0.2365, 0.2286],
        [0.9092, 0.6876, 0.2738, 0.4460],
        [0.4123, 0.0090, 0.7043, 0.3341],
        [0.9092, 0.6876, 0.2738, 0.4460],
        [0.3778, 0.9903, 0.2365, 0.2286]])


## Pytorch中对Tensor进行shuffle
https://blog.csdn.net/qq_31049727/article/details/116206349

直接使用shuffle函数（random.shuffle），但是发现random.shuffle函数会导致Tensor里面数据被重复提取，进而导致输出跟输入分布发生变化。

In [21]:
import torch

valid_sample_ids = torch.arange(12)

# 1、所有elements随机shuffle

t=valid_sample_ids
print(t)
idx = torch.randperm(t.nelement())
t = t.view(-1)[idx].view(t.size())
print(t)


# 2、按照某一特定维度进行shuffle

t=torch.tensor([[1, 2, 3],[3, 4, 5]])
print(t)
idx = torch.randperm(t.shape[1])
t = t[:, idx].view(t.size())
print(t)


tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
tensor([ 5, 10,  8,  7,  1,  0,  9,  3, 11,  6,  4,  2])
tensor([[1, 2, 3],
        [3, 4, 5]])
tensor([[2, 1, 3],
        [4, 3, 5]])


## 4 torch.argmax和torch.where()

### torch.argmax(input, dim, keepdim=False) → LongTensor

作用：返回输入张量中指定维度的最大值的索引。

In [3]:
import torch

# 例1——指定维度：返回相应维度最大值的索引

a= torch.rand(4,4)
print(a)

print(torch.argmax(a, dim=1)) # 列与列之间


# 例2——不指定维度，返回整体上最大值的序号

a = torch.randint(9,(3, 3))
a
# >tensor([[5, 2, 2],
#          [7, 2, 0],
#          [8, 0, 6]])
torch.argmax(a)
# >tensor(6)


tensor([[0.0717, 0.1358, 0.8209, 0.0423],
        [0.5387, 0.5666, 0.5066, 0.9310],
        [0.2416, 0.8954, 0.2820, 0.9936],
        [0.9876, 0.8190, 0.4124, 0.2176]])
tensor([2, 3, 3, 0])


tensor(0)

### torch.where()
```
torch.where(condition, x, y):
condition：判断条件
x：若满足条件，则取x中元素
y：若不满足条件，则取y中元素
```
可以看到torch.where函数会对condition中的元素逐一进行判断，根据判断的结果选取x或y中的值，所以要求x和y应该与condition形状相同。



In [4]:
import torch
 
# 条件
condition = torch.rand(3, 2)
print(condition)
# 满足条件则取x中对应元素
x = torch.ones(3, 2)
print(x)
# 不满足条件则取y中对应元素
y = torch.zeros(3, 2)
print(y)
# 条件判断后的结果
result = torch.where(condition > 0.5, x, y)
print(result)

tensor([[0.9363, 0.3505],
        [0.0822, 0.2145],
        [0.6501, 0.6618]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[1., 0.],
        [0., 0.],
        [1., 1.]])
